In [1]:
import importlib
import funciones_clusters as cl
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
from mne.channels import find_ch_adjacency
from mne.stats import spatio_temporal_cluster_test, ttest_ind_no_p, spatio_temporal_cluster_1samp_test
from functools import partial
%matplotlib qt5

In [35]:
importlib.reload(cl)

<module 'funciones_clusters' from 'c:\\Users\\jhquiza\\OneDrive - Universidad de Medellin\\JupyterNotebooks\\IAT\\funciones_clusters.py'>

In [3]:
# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
rutas_archivos = []
ruta_dir = os.getcwd() + '\\Sin filtrar\\'
dir = os.listdir(ruta_dir)
for q in range(len(dir)):
    ruta_arc =ruta_dir + dir[q]
    rutas_archivos.append(ruta_arc)

evocados_co = {}
evocados_in = {}

for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    epochs = mne.read_epochs(ruta)
    epochs.filter(l_freq=None, h_freq=30)
    evoked_co = epochs['12','13','14','15','16','17','18','19'].average()
    evocados_co[ruta[-14:-9]] = evoked_co
    evoked_in = epochs['24','25','26','27','28','29','30','31'].average()
    evocados_in[ruta[-14:-9]] = evoked_in

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21100n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
145 matching events found
No baseline correction applied
0 projection items activated
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 113 samples (0.441 sec)

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21101n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available

# Cálculo de amplitud media de clústeres - Nueva metodología

In [41]:
# Matrices de muestra de sujetos a analizar
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
negative = list(muestra[muestra['prejudice_level']=='negative']['subject'])
neutral = list(muestra[muestra['prejudice_level']=='neutral']['subject'])
positive = list(muestra[muestra['prejudice_level']=='positive']['subject'])
print(negative)

[23009, 22111, 23013, 21122, 22100, 22116, 22110, 21113, 22113, 21140, 23002, 21139, 23016, 23017, 23006, 21118, 22107, 21123, 22108]


In [20]:
# Clúster 1
# Gráficos clúster toda la época
canales = ['O1', 'P9', 'PO7', 'PO3']
negative_co, neutral_co, positive_co, negative_in, neutral_in, positive_in,negative_dif,neutral_dif,positive_dif, ROI_co, ROI_in, ROI_dif = cl.erps_niveles(canales=canales, evocados_co=evocados_co, evocados_in=evocados_in, positive=positive, negative=negative, neutral=neutral, tmin=0.0, tmax=0.796875)

In [36]:
clusteres = pd.read_csv('Clústeres niveles nueva muestra sin ambos 0.5.csv', index_col='id')
medidas_niveles = cl.batch_medidas_niveles(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, positive=positive, negative=negative, neutral=neutral)
medidas_niveles.head()

,subject,mean_co_1d,mean_in_1d,dif_co_in_1d,mean_co_2d,mean_in_2d,dif_co_in_2d,mean_co_3d,mean_in_3d,dif_co_in_3d,...,dif_co_in_13i,mean_co_14i,mean_in_14i,dif_co_in_14i,mean_co_15i,mean_in_15i,dif_co_in_15i,mean_co_16i,mean_in_16i,dif_co_in_16i
0,23009,-0.090970,-0.204182,0.113211,0.217854,0.343829,-0.125976,0.547694,1.385621,-0.837927,...,0.551056,0.305800,1.233182,-0.927382,0.436521,-0.013322,0.449842,0.468295,0.186783,0.281513
1,22111,1.023734,0.817089,0.206645,2.359010,1.534474,0.824536,0.853590,0.424112,0.429478,...,0.246673,0.692387,0.184015,0.508373,2.792292,1.818420,0.973872,1.308882,1.167924,0.140958
2,23013,0.286197,0.737215,-0.451019,1.041656,-0.277314,1.318970,0.644358,-0.229815,0.874173,...,0.013043,0.131233,0.077348,0.053885,0.066845,-0.034682,0.101527,0.406055,-0.193982,0.600037
3,21122,-1.635589,-1.051160,-0.584428,-1.166454,0.649114,-1.815568,-1.914336,-0.774959,-1.139376,...,0.920466,-1.302394,-1.668112,0.365718,-1.766154,-0.965628,-0.800526,-2.428564,-1.551851,-0.876713
4,22100,-0.818443,-0.856722,0.038279,0.781178,0.055070,0.726108,0.677846,-0.156719,0.834566,...,-0.141699,-0.481067,0.182156,-0.663223,-0.244239,0.277936,-0.522174,-1.711675,-2.346483,0.634808


# Clústeres análisis univariante masivo

In [38]:
clusteres = pd.read_csv('clusters_0.05.csv', index_col='id')
medidas_niveles_um = cl.batch_medidas_niveles(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, positive=positive, negative=negative, neutral=neutral)
medidas_niveles_um.head()

,subject,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2,mean_co_3,mean_in_3,dif_co_in_3
0,23009,0.323383,0.379465,-0.056082,-0.426418,-0.332914,-0.093505,0.223999,-0.470145,0.694144
1,22111,0.947560,0.073099,0.874460,0.955024,0.732380,0.222644,0.961941,0.363269,0.598672
2,23013,0.816437,0.840589,-0.024151,0.034442,0.665648,-0.631206,0.348240,-0.081425,0.429664
3,21122,1.001028,-0.864208,1.865236,-1.843402,-1.339214,-0.504188,-2.335830,-1.706961,-0.628869
4,22100,-1.303307,-1.202247,-0.101060,-0.988424,-0.802932,-0.185491,-1.785529,-2.160402,0.374872


# Análisis tradicional

In [39]:
clusteres = pd.read_csv('ROI NHST.csv', index_col='id')
medidas_niveles_tr = cl.batch_medidas_niveles(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, positive=positive, negative=negative, neutral=neutral)
medidas_niveles_tr.head()

c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:197: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:197: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:197: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:197: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de M

,subject,mean_co_1ci,mean_in_1ci,dif_co_in_1ci,mean_co_2ci,mean_in_2ci,dif_co_in_2ci,mean_co_3ci,mean_in_3ci,dif_co_in_3ci,...,dif_co_in_6d,mean_co_7d,mean_in_7d,dif_co_in_7d,mean_co_8d,mean_in_8d,dif_co_in_8d,mean_co_9d,mean_in_9d,dif_co_in_9d
0,23009,0.122963,0.386481,-0.263519,1.213742,0.943597,0.270146,-0.075322,0.532041,-0.607363,...,-0.401585,1.339968,1.356906,-0.016938,0.274615,-0.249776,0.524391,-0.098663,-0.177807,0.079144
1,22111,0.605126,1.213309,-0.608183,3.904535,2.334682,1.569853,0.628965,0.972501,-0.343535,...,0.363501,1.867144,0.991632,0.875512,1.526488,1.580810,-0.054322,1.543143,1.316748,0.226395
2,23013,0.391237,-0.265045,0.656282,1.417842,0.690605,0.727237,0.655188,0.314050,0.341138,...,1.000149,0.940617,0.334183,0.606434,0.170454,-0.222506,0.392961,0.398807,0.277698,0.121108
3,21122,-0.861500,0.458186,-1.319686,0.381530,0.846152,-0.464622,-1.584082,-1.729462,0.145381,...,-0.833238,-0.937582,-1.056995,0.119413,-1.768140,-1.557471,-0.210669,-1.717080,-0.941553,-0.775527
4,22100,0.003153,-1.349291,1.352443,1.871329,1.553758,0.317571,-0.568300,-0.341454,-0.226845,...,1.778687,0.112361,-0.738227,0.850587,-1.226726,-0.563970,-0.662755,-0.850684,-1.023301,0.172617


In [43]:
muestra.head()

,subject,type,exposure_level,group,victim_self_id,dscore_4,prejudice_level
0,24013,victim,high,victim,yes,-0.039706,neutral
1,23004,excombatant,high,exparamilitar,yes,0.555094,positive
2,24065,victim,high,victim,yes,0.236350,positive
3,23009,excombatant,high,exparamilitar,yes,-0.439115,negative
4,21134,excombatant,high,exparamilitar,yes,-0.099490,neutral


In [42]:
muestra_str = muestra.astype({'subject':'str'}).copy()
todas = pd.merge(muestra_str, medidas_niveles_tr, left_on='subject', right_on='subject', how='inner')
todas.head()

,subject,type,exposure_level,group,victim_self_id,dscore_4,prejudice_level,mean_co_1ci,mean_in_1ci,dif_co_in_1ci,...,dif_co_in_6d,mean_co_7d,mean_in_7d,dif_co_in_7d,mean_co_8d,mean_in_8d,dif_co_in_8d,mean_co_9d,mean_in_9d,dif_co_in_9d
0,24013,victim,high,victim,yes,-0.039706,neutral,0.547537,0.163316,0.384222,...,0.728155,-1.000159,-0.979226,-0.020933,-0.806365,0.557619,-1.363984,1.853521,1.163584,0.689937
1,23004,excombatant,high,exparamilitar,yes,0.555094,positive,0.870352,0.262133,0.608219,...,0.337136,-0.173078,-0.038214,-0.134864,-0.750159,-1.168147,0.417988,0.077534,0.329425,-0.251891
2,24065,victim,high,victim,yes,0.236350,positive,0.605925,0.170672,0.435253,...,0.130723,0.674061,1.572955,-0.898894,-0.482365,1.735003,-2.217369,0.763382,2.136326,-1.372944
3,23009,excombatant,high,exparamilitar,yes,-0.439115,negative,0.122963,0.386481,-0.263519,...,-0.401585,1.339968,1.356906,-0.016938,0.274615,-0.249776,0.524391,-0.098663,-0.177807,0.079144
4,21134,excombatant,high,exparamilitar,yes,-0.099490,neutral,0.125342,0.600217,-0.474875,...,-0.073078,0.364263,0.264410,0.099853,0.242256,-1.014046,1.256302,2.366287,-0.041778,2.408066
